In [44]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [45]:
df = pd.read_excel("../data/raw/litigation_cases.xlsx", skiprows=5, skipfooter=7)
df

,LIT Leave Decision Date - Year,Country of Citizenship,LIT Leave Decision Desc,LIT Case Type Group Desc,LIT Case Type Desc,LIT Filed By Desc,LIT Tribunal Type Desc,LIT Primary Office Type Desc,LIT Primary Office Regional Group Desc,LIT Litigation Count
0,2023,India,Dismissed at Leave,RAD Decisions,RAD-Dismissed-Confirmed-Same,Person Concerned,Federal Court,Inland,IRCC Vancouver Adm,12
1,2021,Fiji,Allowed,IAD Decisions,IAD-Removal Order,Person Concerned,Federal Court,Port of Entry,Pacific Highway District,1
2,2023,Russia,Discontinued - Withdrawn at Leave,Visa Officer Refusal,Visa Officer Decision,Person Concerned,Federal Court,International Network,Unspecified,7
3,2023,Republic of Indonesia,Discontinued - Consent at Leave,HC Decisions,HC Refusal-Perm Res,Person Concerned,Federal Court,Inland,Humanitarian Migration and Integrity Division,2
4,2018,Georgia,Discontinued - Consent at Leave,Visa Officer Refusal,Visa Officer Decision,Person Concerned,Federal Court,International Network,Unspecified,2
...,...,...,...,...,...,...,...,...,...,...
13776,2021,Saudi Arabia,Discontinued - Consent at Leave,Removal Order,Minister-Other / Other Admin.,Person Concerned,Federal Court,Inland,CPC Mississauga,1
13777,2023,People's Republic of China,Discontinued - Consent at Leave,Removal Order,Minister-Other / Other Admin.,Person Concerned,Federal Court,Inland,Montréal Facilitation Center,1
13778,2018,France,Discontinued - Withdrawn at Leave,Visa Officer Refusal,Visa Officer Decision,Person Concerned,Federal Court,Inland,Case Processing Centre Edmonton,1
13779,2023,Angola,Discontinued - Withdrawn at Leave,HC Decisions,HC Refusal-Perm Res,Person Concerned,Federal Court,Inland,Humanitarian Migration and Integrity Division,1


In [46]:
df.isna().sum()

LIT Leave Decision Date - Year            0
Country of Citizenship                    0
LIT Leave Decision Desc                   0
LIT Case Type Group Desc                  0
LIT Case Type Desc                        0
LIT Filed By Desc                         0
LIT Tribunal Type Desc                    0
LIT Primary Office Type Desc              0
LIT Primary Office Regional Group Desc    0
LIT Litigation Count                      0
dtype: int64

In [47]:
for column in df.columns:
    print(f"Value counts for column '{column}':")
    print(df[column].value_counts(dropna=False))
    print("\n")

Value counts for column 'LIT Leave Decision Date - Year':
LIT Leave Decision Date - Year
2021    2641
2019    2582
2022    2536
2018    2356
2023    2105
2020    1561
Name: count, dtype: int64


Value counts for column 'Country of Citizenship':
Country of Citizenship
India                         947
Nigeria                       838
People's Republic of China    662
Pakistan                      497
Mexico                        420
                             ... 
Vanuatu                         1
Luxembourg                      1
Bhutan                          1
Macao SAR                       1
Iceland                         1
Name: count, Length: 180, dtype: int64


Value counts for column 'LIT Leave Decision Desc':
LIT Leave Decision Desc
Dismissed at Leave                   6011
Allowed                              3148
Discontinued - Withdrawn at Leave    2861
Discontinued - Consent at Leave      1454
Allowed - Consent                     260
Not Started at Leave            

In [48]:
import plotly.express as px

# Filter and aggregate top countries by litigation count (2019–2023)
top_countries = df[
    (df["LIT Leave Decision Date - Year"] >= 2018) & 
    (df["LIT Leave Decision Date - Year"] <= 2023)
].groupby("Country of Citizenship")["LIT Litigation Count"].sum().reset_index()

# Create choropleth map
fig = px.choropleth(
    top_countries,
    locations="Country of Citizenship",
    locationmode="country names",
    color="LIT Litigation Count",
    hover_name="Country of Citizenship",
    color_continuous_scale="Reds",
    title="Litigation Count by Country of Citizenship (2019–2023)"
)

fig.update_layout(geo=dict(showframe=False, projection_type='natural earth'))
fig.show()

In [49]:
top_countries = df.groupby("Country of Citizenship")["LIT Litigation Count"].sum().nlargest(10).reset_index()
fig = px.bar(top_countries, x="Country of Citizenship", y="LIT Litigation Count",
             title="Top 10 Countries by Litigation Count", text_auto=True)
fig.show()

In [50]:
litigation_by_year = df.groupby("LIT Leave Decision Date - Year")["LIT Litigation Count"].sum().reset_index()
fig = px.line(litigation_by_year, x="LIT Leave Decision Date - Year", y="LIT Litigation Count",
              title="Litigation Count Over Years", markers=True)
fig.show()

In [51]:
case_type_group = df.groupby("LIT Case Type Group Desc")["LIT Litigation Count"].sum().nlargest(10).reset_index()
fig = px.bar(case_type_group, x="LIT Case Type Group Desc", y="LIT Litigation Count",
             title="Litigation Count by Case Type Groups", text_auto=True)
fig.show()

In [52]:
regional_group = df.groupby("LIT Primary Office Regional Group Desc")["LIT Litigation Count"].sum().nlargest(10).reset_index()
fig = px.bar(regional_group, x="LIT Primary Office Regional Group Desc", y="LIT Litigation Count",
             title="Litigation Count by Regional Group", text_auto=True)
fig.show()

In [53]:
decision_desc = df.groupby("LIT Leave Decision Desc")["LIT Litigation Count"].sum().nlargest().reset_index()

# Calculate total for annotations
total_litigation = decision_desc["LIT Litigation Count"].sum()

# Create the pie chart
fig = px.pie(
    decision_desc,
    names="LIT Leave Decision Desc",
    values="LIT Litigation Count",
    title=f"Distribution of Leave Decision Descriptions (Total = {total_litigation})",
)

# Update text and hover settings
fig.update_traces(
    textinfo="label+percent+value",  # show category, percent, and raw value on the chart
    hovertemplate="<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>"
)

fig.show()

In [54]:
refugee_case_types = [
    "RAD Decisions",
    "RPD Decisions",
    "PRRA",
    "Refugee Eligibility",
    "HC Decisions"
]

years_filter = (df["LIT Leave Decision Date - Year"] >= 2018) & (df["LIT Leave Decision Date - Year"] <= 2023)

df_refugee = df[years_filter & df["LIT Case Type Group Desc"].isin(refugee_case_types)]
df_non_refugee = df[years_filter & ~df["LIT Case Type Group Desc"].isin(refugee_case_types)]

In [55]:
import plotly.express as px

def plot_overall_charts(data, label):
    # 0. Top Countries by Litigation Count (Bar Chart)
    top_countries = data.groupby("Country of Citizenship")["LIT Litigation Count"].sum().nlargest(10).reset_index()
    fig = px.bar(
        top_countries,
        x="Country of Citizenship",
        y="LIT Litigation Count",
        title=f"{label} - Top 10 Countries by Litigation Count",
        text_auto=True
    )
    fig.show()

    # 1. Litigation Count Over Years (Line Chart)
    litigation_by_year = data.groupby("LIT Leave Decision Date - Year")["LIT Litigation Count"].sum().reset_index()
    fig = px.line(
        litigation_by_year,
        x="LIT Leave Decision Date - Year",
        y="LIT Litigation Count",
        title=f"{label} - Litigation Count Over Years",
        markers=True
    )
    fig.show()

    # 2. Litigation Count by Case Type Groups (Bar Chart)
    case_type_group = data.groupby("LIT Case Type Group Desc")["LIT Litigation Count"].sum().nlargest(10).reset_index()
    fig = px.bar(
        case_type_group,
        x="LIT Case Type Group Desc",
        y="LIT Litigation Count",
        title=f"{label} - Litigation Count by Case Type Groups",
        text_auto=True
    )
    fig.show()

    # 3. Litigation Count by Regional Group (Bar Chart)
    regional_group = data.groupby("LIT Primary Office Regional Group Desc")["LIT Litigation Count"].sum().nlargest(10).reset_index()
    fig = px.bar(
        regional_group,
        x="LIT Primary Office Regional Group Desc",
        y="LIT Litigation Count",
        title=f"{label} - Litigation Count by Regional Group",
        text_auto=True
    )
    fig.show()

    # 4. Distribution of Leave Decision Descriptions (Pie Chart)
    decision_desc = data.groupby("LIT Leave Decision Desc")["LIT Litigation Count"].sum().nlargest(5).reset_index()
    total_litigation = decision_desc["LIT Litigation Count"].sum()
    fig = px.pie(
        decision_desc,
        names="LIT Leave Decision Desc",
        values="LIT Litigation Count",
        title=f"{label} - Leave Decision Descriptions (Total = {total_litigation})"
    )
    fig.update_traces(
        textinfo="label+percent+value",
        hovertemplate="<b>%{label}</b><br>Count: %{value}<br>Percentage: %{percent}<extra></extra>"
    )
    fig.show()

In [56]:
plot_overall_charts(df_refugee, "Refugee-Related Cases")

In [57]:

plot_overall_charts(df_non_refugee, "Non-Refugee Cases")

In [58]:
# Optional: restrict years 2019-2023 if needed
years_filter = (df["LIT Leave Decision Date - Year"] >= 2018) & (df["LIT Leave Decision Date - Year"] <= 2023)

top_10_refugee_countries = df_refugee[years_filter].groupby("Country of Citizenship")["LIT Litigation Count"]\
                            .sum().nlargest(12).index.tolist()

top_10_non_refugee_countries = df_non_refugee[years_filter].groupby("Country of Citizenship")["LIT Litigation Count"]\
                                .sum().nlargest(12).index.tolist()
top_10_total_countries = df[years_filter].groupby("Country of Citizenship")["LIT Litigation Count"]\
                                .sum().nlargest(12).index.tolist()

/var/folders/f9/98mwc4pn4l51n7xqzs0w7ryh0000gn/T/ipykernel_37493/3074779142.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/f9/98mwc4pn4l51n7xqzs0w7ryh0000gn/T/ipykernel_37493/3074779142.py:7: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [59]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_refugee_visuals(df_subset, countries, refugee_label):
    # Calculate total litigation count per country to get descending order
    total_litigation = (
        df_subset.groupby("Country of Citizenship")["LIT Litigation Count"]
        .sum()
        .loc[countries]  # limit to given countries
        .sort_values(ascending=False)
    )
    sorted_countries = total_litigation.index.tolist()

    # Filter data for selected countries in descending order
    filtered_df = df_subset[df_subset["Country of Citizenship"].isin(sorted_countries)]

    # --- 1. Line Chart: Litigation Count Over Years (Faceted by Country) ---
    litigation_by_year = (
        filtered_df.groupby(["Country of Citizenship", "LIT Leave Decision Date - Year"])["LIT Litigation Count"]
        .sum().reset_index()
    )

    fig_line = px.line(
        litigation_by_year,
        x="LIT Leave Decision Date - Year",
        y="LIT Litigation Count",
        color="Country of Citizenship",
        category_orders={"Country of Citizenship": sorted_countries},  # enforce order
        facet_col="Country of Citizenship",
        facet_col_wrap=3,
        title=f"{refugee_label} - Litigation Count Over Years by Country",
        markers=True,
        height=900,
        width=1400
    )
    # Make each facet have independent y-axis limits
    fig_line.update_yaxes(matches=None)

    fig_line.update_layout(showlegend=True)
    fig_line.show()

    # --- 2. Individual Pie Charts for Leave Decision Descriptions (Combined) ---
    decision_data = (
        filtered_df.groupby(["Country of Citizenship", "LIT Leave Decision Desc"])["LIT Litigation Count"]
        .sum().reset_index()
    )

    # Only keep the specified top 5 leave decision categories
    top_5_leave_decisions = [
        "Dismissed at Leave",
        "Allowed",
        "Discontinued - Withdrawn at Leave",
        "Discontinued - Consent at Leave",
        "Allowed - Consent"
    ]

    cols = 3
    rows = -(-len(sorted_countries) // cols)  # ceiling division
    fig_pies = make_subplots(
        rows=rows, cols=cols,
        specs=[[{"type": "domain"} for _ in range(cols)] for _ in range(rows)],
        subplot_titles=sorted_countries
    )

    for i, country in enumerate(sorted_countries):
        row = i // cols + 1
        col = i % cols + 1
        
        pie_data = decision_data[
            (decision_data["Country of Citizenship"] == country) &
            (decision_data["LIT Leave Decision Desc"].isin(top_5_leave_decisions))
        ]

        fig_pies.add_trace(
            go.Pie(
                labels=pie_data["LIT Leave Decision Desc"],
                values=pie_data["LIT Litigation Count"],
                name=country,
                textinfo="percent+value",
                showlegend=True
            ),
            row=row, col=col
        )

    fig_pies.update_layout(
        title_text=f"{refugee_label} - Leave Decision Descriptions by Country",
        height=350 * rows,
        width=450 * cols,
        legend_title_text="Leave Decision",
    )

    fig_pies.show()

In [60]:
plot_refugee_visuals(df_refugee, top_10_refugee_countries, refugee_label="Refugee Cases")

In [61]:
plot_refugee_visuals(df_non_refugee, top_10_non_refugee_countries, refugee_label="Non-Refugee Cases")

In [62]:
plot_refugee_visuals(df, top_10_total_countries, refugee_label="Mixed")

In [63]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_case_type_visuals(df_subset, case_types, label):
    # Step 1: Order case types by descending litigation count
    total_litigation = (
        df_subset.groupby("LIT Case Type Group Desc")["LIT Litigation Count"]
        .sum()
        .loc[case_types]
        .sort_values(ascending=False)
    )
    sorted_case_types = total_litigation.index.tolist()

    # Filter dataset
    filtered_df = df_subset[df_subset["LIT Case Type Group Desc"].isin(sorted_case_types)]

    # --- 1. Line Chart: Litigation Count Over Years (Faceted by Case Type) ---
    litigation_by_year = (
        filtered_df.groupby(["LIT Case Type Group Desc", "LIT Leave Decision Date - Year"])["LIT Litigation Count"]
        .sum().reset_index()
    )

    fig_line = px.line(
        litigation_by_year,
        x="LIT Leave Decision Date - Year",
        y="LIT Litigation Count",
        color="LIT Case Type Group Desc",
        category_orders={"LIT Case Type Group Desc": sorted_case_types},
        facet_col="LIT Case Type Group Desc",
        facet_col_wrap=3,
        title=f"{label} - Litigation Count Over Years by Case Type",
        markers=True,
        height=900,
        width=1400
    )
    fig_line.update_yaxes(matches=None)
    fig_line.update_layout(showlegend=True)
    fig_line.show()

    # --- 2. Pie Charts for Leave Decision Descriptions (Top 5 only) ---
    decision_data = (
        filtered_df.groupby(["LIT Case Type Group Desc", "LIT Leave Decision Desc"])["LIT Litigation Count"]
        .sum().reset_index()
    )

    top_5_leave_decisions = [
        "Dismissed at Leave",
        "Allowed",
        "Discontinued - Withdrawn at Leave",
        "Discontinued - Consent at Leave",
        "Allowed - Consent"
    ]

    cols = 3
    rows = -(-len(sorted_case_types) // cols)
    fig_pies = make_subplots(
        rows=rows, cols=cols,
        specs=[[{"type": "domain"} for _ in range(cols)] for _ in range(rows)],
        subplot_titles=sorted_case_types
    )

    for i, case_type in enumerate(sorted_case_types):
        row = i // cols + 1
        col = i % cols + 1

        pie_data = decision_data[
            (decision_data["LIT Case Type Group Desc"] == case_type) &
            (decision_data["LIT Leave Decision Desc"].isin(top_5_leave_decisions))
        ]

        fig_pies.add_trace(
            go.Pie(
                labels=pie_data["LIT Leave Decision Desc"],
                values=pie_data["LIT Litigation Count"],
                name=case_type,
                textinfo="percent+value",
                showlegend=True
            ),
            row=row, col=col
        )

    fig_pies.update_layout(
        title_text=f"{label} - Leave Decision Descriptions by Case Type",
        height=350 * rows,
        width=450 * cols,
        legend_title_text="Leave Decision",
    )

    fig_pies.show()

In [64]:
# Get top 10 case types by litigation count (from 2018 to 2023, optionally)
years_filter = (df["LIT Leave Decision Date - Year"] >= 2018) & (df["LIT Leave Decision Date - Year"] <= 2023)
top_case_types = df[years_filter].groupby("LIT Case Type Group Desc")["LIT Litigation Count"]\
                    .sum().nlargest(10).index.tolist()

# Call the function
plot_case_type_visuals(df[years_filter], top_case_types, label="All Cases")

In [65]:
import plotly.express as px

def plot_leave_decision_percentage_trends(df, label):
    # Top 5 leave decision types
    top_decision_types = [
        "Dismissed at Leave",
        "Allowed",
        "Discontinued - Withdrawn at Leave",
        "Discontinued - Consent at Leave"
    ]
    
    # Filter data to include only top 5 decisions
    df_decision = df[df["LIT Leave Decision Desc"].isin(top_decision_types)]

    # Get top 9 countries by total litigation count
    top_countries = df_decision.groupby("Country of Citizenship")["LIT Litigation Count"]\
        .sum().nlargest(9).index.tolist()

    # Filter for those countries
    filtered_df = df_decision[df_decision["Country of Citizenship"].isin(top_countries)]

    # Sort countries by total litigation count for consistent facet order
    sorted_countries = filtered_df.groupby("Country of Citizenship")["LIT Litigation Count"]\
        .sum().sort_values(ascending=False).index.tolist()

    # Group by year, country, and decision
    grouped = filtered_df.groupby(
        ["Country of Citizenship", "LIT Leave Decision Date - Year", "LIT Leave Decision Desc"]
    )["LIT Litigation Count"].sum().reset_index()

    # Total litigation count per country-year for percentage calculation
    total_per_year_country = grouped.groupby(
        ["Country of Citizenship", "LIT Leave Decision Date - Year"]
    )["LIT Litigation Count"].sum().reset_index().rename(columns={"LIT Litigation Count": "Total Count"})

    # Merge total with grouped to compute percentage
    merged = grouped.merge(
        total_per_year_country,
        on=["Country of Citizenship", "LIT Leave Decision Date - Year"]
    )
    merged["Percentage"] = (merged["LIT Litigation Count"] / merged["Total Count"]) * 100

    # Plot faceted line chart with percentage
    fig = px.line(
        merged,
        x="LIT Leave Decision Date - Year",
        y="Percentage",
        color="Country of Citizenship",
        facet_col="LIT Leave Decision Desc",
        facet_col_wrap=2,
        category_orders={"Country of Citizenship": sorted_countries},
        title=f"{label} - Percentage Trends of Leave Decision Types by Top Countries",
        markers=True,
        height=1000,
        width=1400
    )

    fig.update_yaxes(matches=None, title="Percentage (%)")
    fig.update_layout(
        showlegend=True,
        legend_title="Leave Decision Type"
    )

    fig.show()


In [66]:
plot_leave_decision_percentage_trends(df, label="All")

In [67]:
import plotly.express as px
import pandas as pd

def plot_country_treemaps(df, label):
    # Top 5 decision types
    top_decision_types = [
        "Dismissed at Leave",
        "Allowed",
        "Discontinued - Withdrawn at Leave",
        "Discontinued - Consent at Leave"
    ]

    # Filter to include only top 5 decision types
    df_filtered = df[df["LIT Leave Decision Desc"].isin(top_decision_types)]

    # Get top 10 countries by litigation count
    top_countries = df_filtered.groupby("Country of Citizenship")["LIT Litigation Count"]\
        .sum().nlargest(10).index.tolist()

    df_filtered = df_filtered[df_filtered["Country of Citizenship"].isin(top_countries)]

    # Get top 10 case types by litigation count
    top_case_types = df_filtered.groupby("LIT Case Type Group Desc")["LIT Litigation Count"]\
        .sum().nlargest(10).index.tolist()

    df_filtered = df_filtered[df_filtered["LIT Case Type Group Desc"].isin(top_case_types)]

    # Generate a treemap for each country
    for country in top_countries:
        df_country = df_filtered[df_filtered["Country of Citizenship"] == country]

        grouped = df_country.groupby([
            "LIT Case Type Group Desc", "LIT Leave Decision Desc"
        ])["LIT Litigation Count"].sum().reset_index()

        fig = px.treemap(
            grouped,
            path=["LIT Case Type Group Desc", "LIT Leave Decision Desc"],
            values="LIT Litigation Count",
            color="LIT Leave Decision Desc",
            color_discrete_sequence=px.colors.qualitative.Set2,
            title=f"{label} - Treemap of Leave Decisions by Case Type ({country})",
            height=700,
            width=1000
        )

        fig.update_traces(root_color="lightgrey")
        fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
        fig.show()


In [68]:
plot_country_treemaps(df, label="Total Cases")

In [69]:
def plot_sankey(df):
    # Normalize "Discontinued" variations
    df["LIT Leave Decision Desc"] = df["LIT Leave Decision Desc"].replace({
        "Discontinued - Withdrawn at Leave": "Discontinued",
        "Discontinued - Consent at Leave": "Discontinued"
    })

    # Filter selected leave decisions
    valid_decisions = ["Dismissed at Leave", "Allowed", "Discontinued"]
    df = df[df["LIT Leave Decision Desc"].isin(valid_decisions)]

    # Filter selected case types
    valid_case_types = ["RAD Decisions", "Visa Officer Refusal", "Mandamus"]
    df = df[df["LIT Case Type Group Desc"].isin(valid_case_types)]

    # Top 4 countries by count
    top_countries = df.groupby("Country of Citizenship")["LIT Litigation Count"].sum().nlargest(4).index.tolist()
    df = df[df["Country of Citizenship"].isin(top_countries)]

    # Create label list
    countries = df["Country of Citizenship"].unique().tolist()
    case_types = df["LIT Case Type Group Desc"].unique().tolist()
    decisions = df["LIT Leave Decision Desc"].unique().tolist()
    labels = countries + case_types + decisions

    # Label to index mapping
    label_map = {label: idx for idx, label in enumerate(labels)}

    # Group: Country → Case Type
    flows1 = df.groupby(["Country of Citizenship", "LIT Case Type Group Desc"])["LIT Litigation Count"].sum().reset_index()
    # Group: Case Type → Leave Decision
    flows2 = df.groupby(["LIT Case Type Group Desc", "LIT Leave Decision Desc"])["LIT Litigation Count"].sum().reset_index()

    # Combine source-target-value
    source = flows1["Country of Citizenship"].map(label_map).tolist() + flows2["LIT Case Type Group Desc"].map(label_map).tolist()
    target = flows1["LIT Case Type Group Desc"].map(label_map).tolist() + flows2["LIT Leave Decision Desc"].map(label_map).tolist()
    values = flows1["LIT Litigation Count"].tolist() + flows2["LIT Litigation Count"].tolist()

    # Compute percentages
    total = sum(values)
    percentages = [(v / total) * 100 for v in values]
    
    # Create visible flow labels as percentages
    flow_labels = [f"{p:.1f}%" for p in percentages]

    # Create Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=20,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=labels
        ),
        link=dict(
            source=source,
            target=target,
            value=values,
            label=flow_labels,
            hovertemplate="%{source.label} → %{target.label}<br>%{value} cases (%{label})"
        )
    )])

    fig.update_layout(
        title_text="Sankey Diagram: Country → Case Type → Leave Decision",
        font_size=12,
        height=800,
        width=1000
    )
    fig.show()

In [70]:
plot_sankey(df)

In [71]:
import plotly.express as px

def plot_animated_time_series(df):
    top_decisions = [
        "Dismissed at Leave", "Allowed", 
        "Discontinued - Withdrawn at Leave", 
        "Discontinued - Consent at Leave"
    ]
    df = df[df["LIT Leave Decision Desc"].isin(top_decisions)]

    top_countries = df.groupby("Country of Citizenship")["LIT Litigation Count"].sum().nlargest(10).index.tolist()
    df = df[df["Country of Citizenship"].isin(top_countries)]

    df_grouped = df.groupby([
        "LIT Leave Decision Date - Year", "Country of Citizenship", "LIT Leave Decision Desc"
    ])["LIT Litigation Count"].sum().reset_index()

    # Sort countries descending by total litigation count for consistent ordering
    country_order = df_grouped.groupby("Country of Citizenship")["LIT Litigation Count"].sum()\
                    .sort_values(ascending=False).index.tolist()
    
    # Calculate max total litigation count per country-year for y-axis limit
    max_y = df_grouped.groupby(["LIT Leave Decision Date - Year", "Country of Citizenship"])["LIT Litigation Count"]\
            .sum().max()
    y_max = max_y * 1.1  # add 10% padding

    fig = px.bar(
        df_grouped,
        x="Country of Citizenship",
        y="LIT Litigation Count",
        color="LIT Leave Decision Desc",
        animation_frame="LIT Leave Decision Date - Year",
        title="Litigation Decisions Over Years by Country",
        barmode="stack",
        category_orders={"Country of Citizenship": country_order},
        height=700,
        width=1300
    )

    # Fix y-axis range
    fig.update_yaxes(range=[0, y_max])

    fig.update_layout(
        margin=dict(l=50, r=50, t=100, b=150),
        xaxis=dict(
            tickangle=-45,
            tickfont=dict(size=10),
            tickmode='array',
            tickvals=country_order,
            ticktext=country_order
        ),
        legend=dict(font=dict(size=12)),
        updatemenus=[{
            "buttons": [{
                "args": [None, {
                    "frame": {"duration": 1200, "redraw": True},
                    "fromcurrent": True,
                    "transition": {"duration": 600, "easing": "linear"}
                }],
                "label": "play",
                "method": "animate"
            }],
            "type": "buttons",
            "showactive": False
        }]
    )

    fig.show()

In [72]:
plot_animated_time_series(df)

In [73]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# List of unique case types to loop through
case_types = df["LIT Case Type Group Desc"].dropna().unique().tolist()

for case in case_types:
    print(f"Generating visualizations for {case}...")

    # Filter data for the current case type
    case_df = df[df["LIT Case Type Group Desc"] == case]

    # Get top 5 countries by total litigation count
    top_countries = (
        case_df.groupby("Country of Citizenship")["LIT Litigation Count"]
        .sum()
        .nlargest(10)
        .index
    )
    df_top = case_df[case_df["Country of Citizenship"].isin(top_countries)]

    # Grouped Bar Chart: Litigation count by country
    grouped = (
        df_top.groupby("Country of Citizenship")["LIT Litigation Count"]
        .sum()
        .reset_index()
        .sort_values("LIT Litigation Count", ascending=False)
    )

    fig_bar = go.Figure(
        data=[
            go.Bar(
                x=grouped["Country of Citizenship"],
                y=grouped["LIT Litigation Count"],
                text=grouped["LIT Litigation Count"],
                textposition="outside",
                marker_color="indianred"
            )
        ]
    )

    fig_bar.update_layout(
        title=f"{case} Litigation Counts by Country (Top 5)",
        xaxis_title="Country of Citizenship",
        yaxis_title="Litigation Count",
        width=1000,
        height=600
    )

    fig_bar.show()

    # Time Series Line Chart: Yearly trends by country
    time_grouped = (
        df_top.groupby(["LIT Leave Decision Date - Year", "Country of Citizenship"])["LIT Litigation Count"]
        .sum()
        .reset_index()
    )

    fig_ts = px.line(
        time_grouped,
        x="LIT Leave Decision Date - Year",
        y="LIT Litigation Count",
        color="Country of Citizenship",
        markers=True,
        title=f"{case} Litigation Trends Over Time (Top 5 Countries)"
    )

    fig_ts.update_layout(
        xaxis_title="Year",
        yaxis_title="Litigation Count",
        height=700,
        width=1200
    )

    fig_ts.show()

Generating visualizations for RAD Decisions...


Generating visualizations for IAD Decisions...


Generating visualizations for Visa Officer Refusal...


Generating visualizations for HC Decisions...


Generating visualizations for PRRA...


...

Generating visualizations for In-Canada Classes...


Generating visualizations for Refugee Eligibility...


Generating visualizations for Detention...


Generating visualizations for DFN...


Generating visualizations for Danger Opinions...
